In [13]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3243, -0.0306,  0.3082, -0.0317,  0.0985, -0.1333, -0.4458,  0.1788,
          0.1287, -0.0755,  0.0601, -0.6446,  0.4938,  0.6412,  0.1737, -0.3207,
          0.5754, -0.2947,  0.5602, -0.3085],
        [ 0.0152,  0.4772,  0.4580,  0.2044, -0.4078, -0.2823, -0.1938, -0.1358,
          0.1499,  0.4159, -0.2317, -0.9016,  0.5427,  0.2279,  0.1181, -0.2269,
          0.2674, -0.0077,  0.2380, -0.3112],
        [ 0.1605,  0.0474,  0.1699, -0.1971, -0.2438, -0.1527, -0.2100, -0.2891,
          0.1549,  0.4221, -0.1009, -0.5560,  0.6460, -0.0027,  0.0630, -0.2082,
          0.1115, -0.3417,  0.2971, -0.2509]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.3082, 0.0000, 0.0985, 0.0000, 0.0000, 0.1788, 0.1287,
         0.0000, 0.0601, 0.0000, 0.4938, 0.6412, 0.1737, 0.0000, 0.5754, 0.0000,
         0.5602, 0.0000],
        [0.0152, 0.4772, 0.4580, 0.2044, 0.0000, 0.0000, 0.0000, 0.0000, 0.1499,
         0.4159, 0.0000, 0.0000, 0.5427, 0.2279, 0.11

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0319, -0.0154,  0.0031,  ..., -0.0102,  0.0314,  0.0058],
        [-0.0159,  0.0043,  0.0132,  ..., -0.0109,  0.0014,  0.0254]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0033, -0.0094], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0060, -0.0428, -0.0401,  ...,  0.0387, -0.0020, -0.0088],
        [ 0.0337,  0.0337,  0.0233,  ...,  0.0425, -0.0028, -0.0026]],
       device='cuda:0', grad_fn=<Sl